In [1]:
# @name
# @description
# @author
# @date

# Description

This is the notebook for the creation of the first review network and derived hypotheses. 

* Reading intermediary variables from files. In this workflow variables are first output as files and read from these files for the next step. This attribute is convenient when some functions require several hours of runtime.


* Review network: From Monarch knowledge graph, we built a network seeded by 8 nodes, retrieving their explicit relationships and all the relationships among all these nodes. Seed nodes:

    - 'MONDO:0014109', # NGLY1 deficiency
    - 'HGNC:17646', # NGLY1 human gene
    - 'HGNC:633', # AQP1 human gene
    - 'MGI:103201', # AQP1 mouse gene
    - 'HGNC:7781', # NRF1 human gene* Ginger: known as NFE2L1. http://biogps.org/#goto=genereport&id=4779
    - 'HGNC:24622', # ENGASE human gene
    - 'HGNC:636', # AQP3 human gene
    - 'HGNC:19940' # AQP11 human gene
    

* Connecting paths: query templates.

In [1]:
#import time
import transcriptomics, regulation, curation, monarch, graph, neo4jlib, hypothesis, summary

## Edges library
### Review edges to integrate into the knowledge graph
#### import transcriptomics
We retrieved edges from RNA-seq transcriptomics profiles using the transcriptomics module:

    - Experimental data sets: from Chow et al. paper [pmid:29346549] (NGLY1 deficiency model on fruit fly)

###### At once

In [3]:
%%time
# prepare data to graph schema
data = transcriptomics.read_data()
transcriptomics.clean_data(data)
transcriptomics.prepare_data_edges()
rna_edges = transcriptomics.prepare_rna_edges()

# build network
transcriptomics.build_edges(rna_edges)
transcriptomics.build_nodes(rna_edges)


* This is the size of the raw expression data structure: (15370, 9)
* These are the expression attributes: Index(['FlyBase ID', 'Symbol', 'baseMean', 'log2FoldChange', 'Unnamed: 4',
       'lfcSE', 'stat', 'pvalue', 'padj'],
      dtype='object')
* This is the first record:
    FlyBase ID  Symbol    baseMean  log2FoldChange  Unnamed: 4     lfcSE  \
0  FBgn0030880  CG6788  175.577087       -4.209283     0.05406  0.190308   

        stat         pvalue           padj  
0 -22.118249  2.110000e-108  2.860000e-104  

* This is the size of the clean expression data structure: (386, 6)
* These are the clean expression attributes: Index(['FlyBase ID', 'Symbol', 'log2FoldChange', 'pvalue', 'padj',
       'Regulation'],
      dtype='object')
* This is the first record:
    FlyBase ID Symbol  log2FoldChange        pvalue      padj   Regulation
0  FBgn0035904  GstO3        0.576871  2.130000e-08  0.000002  Upregulated

* This is the size of the expression data structure: (386, 13)
* These are th

Network is returned as both CSV files at graph/ and digital object

#### import regulation

###### At once

In [4]:
%%time
# prepare msigdb data
regulation.prepare_msigdb_data()

# prepare individual networks
data = regulation.load_tf_gene_edges()
dicts = regulation.get_gene_id_normalization_dictionaries(data)
regulation.prepare_data_edges(data, dicts)

# prepare regulation network
regulation_edges = regulation.prepare_regulation_edges()

# build network
regulation.build_edges(regulation_edges)
regulation.build_nodes(regulation_edges)


* Number of Transcription Factor Targets (TFT) gene sets: 615

* Querying BioThings to map gene symbols to hgnc and entrez IDs...
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-3071...done.
Finished.
53 input query terms found no hit:
	['TFIII', 'CREL', 'GNCF', 'T3R', 'E2F4DP2', 'TAL1BETAE47', 'AHRARNT', 'FOX', 'COREBINDINGFACTOR', 'C
Pass "returnall=True" to return complete lists of duplicate or missing query terms.

* Querying BioThings to map entrez to hgnc IDs and gene symbols...
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-16632...

Network is returned as both CSV file at graph/ and digital object

#### import curation

###### At once

In [5]:
%%time
# prepare curated edges and nodes
# from file:
edges_df, nodes_df = curation.read_data()
curated_edges = curation.prepare_curated_edges(edges_df)
curated_nodes = curation.prepare_curated_nodes(nodes_df)

# build edges and nodes files
curation.build_edges(curated_edges)
curation.build_nodes(curated_nodes)


 Read data from curation/data/v2018 dir...

* Number of curated edges: 321

* Number of curated nodes: 288

ID conversion: from ngly1 curated network to graph schema...

Mapping genes to HGNC ID...
querying 1-18...done.
Finished.

Adding diseases to MONDO ID network...

Adding gene to protein network...
querying 1-43...done.
Finished.

Drop duplicated gene-protein relations...

Mapping genes to HGNC ID...
querying 1-18...done.
Finished.

Adding diseases described by the MONDO ontology...

Adding Name attribute: gene names from BioThings...
querying 1-298...done.
Finished.
38 input query terms found dup hits:
	[('or', 5), ('NGLY1', 3), ('of', 16), ('1', 14), ('by', 9), ('MRS', 9), ('CSF', 8), ('acid', 4), ('B
591 input query terms found no hit:
	['NGLY1-deficiency', 'misfolded', 'incompletely', 'synthesized', 'protein', 'catabolic', 'process', 
Pass "returnall=True" to return complete lists of duplicate or missing query terms.

Preparing encoding genes from ngly1 curated network...

Ad

Network is returned as both CSV file at graph/ and digital object

#### import monarch
We retrieved edges from Monarch using the monarch module:

    - From 8 seed nodes we retrieved 1st shell
    - From all seed and 1 shell nodes we retrieved edges among them

In [6]:
%%time
# prepare data to graph schema
# seed nodes
seedList = [ 
    'MONDO:0014109', # NGLY1 deficiency
    'HGNC:17646', # NGLY1 human gene
    'HGNC:633', # AQP1 human gene
    'MGI:103201', # AQP1 mouse gene
    'HGNC:7781', # NRF1 human gene* Ginger: known as NFE2L1. http://biogps.org/#goto=genereport&id=4779
    'HGNC:24622', # ENGASE human gene
    'HGNC:636', # AQP3 human gene
    'HGNC:19940' # AQP11 human gene
] 

# get first shell of neighbours
neighboursList = monarch.get_neighbours_list(seedList)
print(len(neighboursList))


The function "get_neighbours_list()" is running, please keep calm and have some coffee...


100%|██████████| 8/8 [00:21<00:00,  2.03s/it]


Finished get_neighbours...
705
CPU times: user 589 ms, sys: 40.3 ms, total: 629 ms
Wall time: 21.6 s


In [7]:
%%time
# introduce animal model ortho-phenotypes for seed and 1st shell neighbors
seed_orthophenoList = monarch.get_orthopheno_list(seedList)
print(len(seed_orthophenoList))


The function "get_orthopheno_list()" is running, please keep calm and have some coffee...


100%|██████████| 8/8 [00:18<00:00,  1.86s/it]


Finished get_neighbours...


100%|██████████| 82/82 [01:21<00:00,  1.15it/s]


Finished get_neighbours...
240
CPU times: user 5.12 s, sys: 296 ms, total: 5.41 s
Wall time: 1min 40s


In [8]:
%%time
neighbours_orthophenoList = monarch.get_orthopheno_list(neighboursList)
print(len(neighbours_orthophenoList))


The function "get_orthopheno_list()" is running, please keep calm and have some coffee...


100%|██████████| 705/705 [1:24:24<00:00, 17.10s/it]  


Finished get_neighbours...


100%|██████████| 1497/1497 [32:16<00:00,  1.17s/it] 


Finished get_neighbours...
4411
CPU times: user 2min 58s, sys: 11.9 s, total: 3min 10s
Wall time: 1h 56min 41s


In [9]:
%%time
# network nodes: seed + 1shell + ortholog-phentoype
geneList = sum([seedList,
                neighboursList,
                seed_orthophenoList,
                neighbours_orthophenoList], 
               [])
print('genelist: ',len(geneList))

# get edge expansion
network = monarch.extract_edges(geneList)
print('network: ',len(network))

genelist:  5364

The function "expand_edges()" is running, please keep calm and have some coffee...


100%|██████████| 5086/5086 [5:06:44<00:00, 13.70s/it]   


Finished get_connections...
network:  36235
CPU times: user 7min 21s, sys: 28.6 s, total: 7min 49s
Wall time: 5h 6min 44s


monarch data return as digital object and, if print_network(), as CSV file at monarch/

In [10]:
%%time
# save network
monarch.print_network(network, 'monarch_connections')


File '/home/nuria/workspace/graph-hypothesis-generation-lib/plan/monarch/monarch_connections_v2019-03-04.csv' saved.
CPU times: user 379 ms, sys: 7.57 ms, total: 386 ms
Wall time: 382 ms


In [11]:
%%time
# build network
file_name = 'monarch_connections_v2019-03-04.csv'
monarch_connections = monarch.read_connections(file_name)
monarch.build_edges(monarch_connections)
monarch.build_nodes(monarch_connections)


* This is the size of the data structure: (36235, 7)
* These are the attributes: Index(['object_id', 'object_label', 'reference_id_list', 'relation_id',
       'relation_label', 'subject_id', 'subject_label'],
      dtype='object')
* This is the first record:
     object_id object_label reference_id_list relation_id  relation_label  \
0  MGI:1345634         Amfr               NaN  RO:0002434  interacts with   

    subject_id subject_label  
0  MGI:1915069         Derl1  
df (36235, 9)

* This is the size of the edges file data structure: (36235, 9)
* These are the edges attributes: Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
* This is the first record:
     object_id property_description property_id  property_label  \
0  MGI:1345634                   NA  RO:0002434  interacts with   

                                proper

In [13]:
monarch.print_nodes(geneList,'monarch_seed_nodes')


File '/home/nuria/workspace/graph-hypothesis-generation-lib/plan/monarch/monarch_seed_nodes_v2019-03-04.csv' saved.


In [23]:
## CHECK ##
# why seed geneList (5364) < monarch nodes (5086)?
# Answer: because i convert it to set() in the extract_edges() method 
import pandas as pd
s = set(geneList)
n_df = pd.read_csv('monarch/monarch_nodes_v2019-03-04.csv')
n = set(n_df.id)
print('geneList length:', len(geneList))
print('geneList without duplicates:', len(s))
print('monarch nodesList:', len(n))

geneList length: 5364
geneList without duplicates: 5086
monarch nodesList: 5086


Network is returned as both CSV file and digital object

###### At once

In [ ]:
%%time
# prepare data to graph schema
# seed nodes
seedList = [ 
    'MONDO:0014109', # NGLY1 deficiency
    'HGNC:17646', # NGLY1 human gene
    'HGNC:633', # AQP1 human gene
    'MGI:103201', # AQP1 mouse gene
    'HGNC:7781', # NRF1 human gene* Ginger: known as NFE2L1. http://biogps.org/#goto=genereport&id=4779
    'HGNC:24622', # ENGASE human gene
    'HGNC:636', # AQP3 human gene
    'HGNC:19940' # AQP11 human gene
] 

# get first shell of neighbours
neighboursList = monarch.get_neighbours_list(seedList)
print(len(neighboursList))

# introduce animal model ortho-phenotypes for seed and 1st shell neighbors
seed_orthophenoList = monarch.get_orthopheno_list(seedList)
print(len(seed_orthophenoList))
neighbours_orthophenoList = monarch.get_orthopheno_list(neighboursList)
print(len(neighbours_orthophenoList))

# network nodes: seed + 1shell + ortholog-phentoype
geneList = sum([seedList,
                neighboursList,
                seed_orthophenoList,
                neighbours_orthophenoList], 
               [])
print('genelist: ',len(geneList))

# get edge expansion
network = monarch.extract_edges(geneList)
print('network: ',len(network))

# save edges
monarch.print_network(network, 'monarch_connections')

# build network
file_name = 'monarch_connections_v2019-03-04.csv'
monarch_connections = monarch.read_connections(file_name)
monarch.build_edges(monarch_connections)
monarch.build_nodes(monarch_connections)

Network is returned as both CSV file and digital object

## Graph library
### Create the review knowledge graph
#### import graph

Tasks:

* Load Networks and calculate graph nodes
* Monarch graph connectivity
* Build graph

In [3]:
%%time
# load networks and calculate graph nodes
curation_file = './graph/curated_graph_edges_v2019-03-04.csv'
monarch_file = './monarch/monarch_edges_v2019-03-04.csv'
rna_file = './graph/rna_edges_v2019-03-04.csv'
tf_file = './graph/regulation_edges_v2019-03-04.csv'
graph_nodes_list = graph.graph_nodes(
    curation_file_path=curation_file,
    monarch_file_path=monarch_file,
    transcriptomics_file_path=rna_file,
    regulation_file_path=tf_file
)


/home/nuria/anaconda3/envs/avalanche/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


Curated:
(362, 10)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri', 'g2p_mark'],
      dtype='object')
Monarch:
(36235, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Transcriptomics:
(386, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
Regulatory:
(197267, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')

Concatenating into a graph...
(36983, 9)

Drop duplicated rows...


In [5]:
## CHECK ##
print('graph nodes list length:', len(graph_nodes_list))
print('graph nodes set length:', len(set(graph_nodes_list)))

graph nodes list length: 9809
graph nodes set length: 9809


In [6]:
%%time
# monarch graph connectivity
# get edge expansion
network = monarch.extract_edges(graph_nodes_list)
print('network: ',len(network))


The function "extract_edges()" is running, please keep calm and have some coffee...


100%|██████████| 9809/9809 [10:04:36<00:00,  5.29s/it]  


Finished get_connections...
network:  236506
CPU times: user 14min 22s, sys: 52.9 s, total: 15min 15s
Wall time: 10h 4min 36s


In [7]:
# save network
monarch.print_network(network, 'monarch_connections_graph')


File '/home/nuria/workspace/graph-hypothesis-generation-lib/plan/monarch/monarch_connections_graph_v2019-03-05.csv' saved.


In [8]:
%%time
# build network
file_name = 'monarch_connections_graph_v2019-03-05.csv'
monarch_connections = monarch.read_connections(file_name)
monarch.build_edges(monarch_connections)
monarch.build_nodes(monarch_connections)


* This is the size of the data structure: (236506, 7)
* These are the attributes: Index(['object_id', 'object_label', 'reference_id_list', 'relation_id',
       'relation_label', 'subject_id', 'subject_label'],
      dtype='object')
* This is the first record:
   object_id object_label reference_id_list relation_id  relation_label  \
0  HGNC:3236         EGFR               NaN  RO:0002434  interacts with   

  subject_id subject_label  
0  HGNC:7553           MYC  
df (236506, 9)

* This is the size of the edges file data structure: (236506, 9)
* These are the edges attributes: Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
* This is the first record:
   object_id property_description property_id  property_label  \
0  HGNC:3236                   NA  RO:0002434  interacts with   

                                property_uri re

In [ ]:
# build graph
edges = graph.build_edges()

In [ ]:
nodes = graph.build_nodes(edges)

###### At once

In [ ]:
%%time
# build graph
edges = graph.build_edges()
nodes = graph.build_nodes(edges)

## Neo4jlib library
### Import the graph into Neo4j graph database
#### import neo4jlib

In [ ]:
# import to graph interface, by now neo4j
## get edges and files for neo4j
edges_df = neo4jlib.get_dataframe(edges)

In [ ]:
nodes_df = neo4jlib.get_dataframe(nodes)

In [ ]:
statements = neo4jlib.get_statements(edges_df)

In [ ]:
concepts = neo4jlib.get_concepts(nodes_df)

In [ ]:
## import the graph into neo4j
# save files into neo4j import dir
neo4j_path = '/home/nuria/ngly1-graph/neo4j-graphs/neo4j/neo4j-lib'
#neo4j_path = './neo4j-community-3.0.3'
neo4jlib.save_neo4j_files(statements, neo4j_path, file_type = 'statements')

In [ ]:
neo4jlib.save_neo4j_files(concepts, neo4j_path, file_type = 'concepts')

In [ ]:
# import graph into neo4j
neo4jlib.do_import(neo4j_path)

###### At once

In [ ]:
%%time
# import to graph interface, by now neo4j
## get edges and files for neo4j
edges_df = neo4jlib.get_dataframe(edges)
nodes_df = neo4jlib.get_dataframe(nodes)
statements = neo4jlib.get_statements(edges_df)
concepts = neo4jlib.get_concepts(nodes_df)
print('statements: ', len(statements))
print('concepts: ',len(concepts))

## import the graph into neo4j
# save files into neo4j import dir
neo4j_path = '/home/nuria/ngly1-graph/neo4j-graphs/neo4j/neo4j-lib'
#neo4j_path = './neo4j-community-3.0.3'
neo4jlib.save_neo4j_files(statements, neo4j_path, file_type = 'statements')
neo4jlib.save_neo4j_files(concepts, neo4j_path, file_type = 'concepts')

# import graph into neo4j
neo4jlib.do_import(neo4j_path)

=> Alternatively from file:

In [ ]:
%%time
import neo4jlib
# import to graph interface, by now neo4j
## get edges and files for neo4j
edges = neo4jlib.get_dataframe_from_file('./graph/graph_edges_v2019-02-22')
nodes = neo4jlib.get_dataframe_from_file('./graph/graph_nodes_v2019-02-22')
statements = neo4jlib.get_statements(edges)
concepts = neo4jlib.get_concepts(nodes)
print('statements: ', len(statements))
print('concepts: ',len(concepts))

## import the graph into neo4j
# save files into neo4j import dir
neo4j_path = './neo4j-community-3.0.3'
neo4jlib.save_neo4j_files(statements, neo4j_path, file_type='statements')
neo4jlib.save_neo4j_files(concepts, neo4j_path, file_type='concepts')

# import graph into neo4j
neo4jlib.do_import(neo4j_path)

## hypothesis-generation library
### Query the graph for mechanistic explanation, then summarize the extracted paths
#### import hypothesis, summary

### Ortopheno query with general nodes/relations removed
#### import the graph into neo4j from file

In [ ]:
import neo4jlib, hypothesis, summary

In [ ]:
%%time
# import to graph interface, by now neo4j
## get edges and files for neo4j
edges_df = neo4jlib.get_dataframe_from_file('./graph/graph_edges_v2019-02-22')
nodes_df = neo4jlib.get_dataframe_from_file('./graph/graph_nodes_v2019-02-22')
statements = neo4jlib.get_statements(edges_df)
concepts = neo4jlib.get_concepts(nodes_df)
print('statements: ', len(statements))
print('concepts: ',len(concepts))

## import the graph into neo4j
# save files into neo4j import dir
neo4j_path = './neo4j-community-3.0.3'
neo4jlib.save_neo4j_files(statements, neo4j_path, file_type = 'statements')
neo4jlib.save_neo4j_files(concepts, neo4j_path, file_type = 'concepts')

# import graph into neo4j
neo4jlib.do_import(neo4j_path)

In [ ]:
%%time
# get orthopheno paths
seed = list([
        'HGNC:17646',  # NGLY1 human gene
        'HGNC:633'  # AQP1 human gene
])
hypothesis.query(seed,queryname='ngly1_aqp1',port='7687') #http_port= 7470; bolt_port=7680

Checked manually that there are paths

In [ ]:
%%time
# get orthopheno paths
seed = list([
        'HGNC:17646',  # NGLY1 human gene
        'HGNC:633'  # AQP1 human gene
])
hypothesis.query(seed, queryname='ngly1_aqp1', pwdegree='1000', phdegree='1000', port='7687')

Checked manually that there are results. 

**There is not a check if there are results diff to 0**

In [ ]:
%%time
import hypothesis
# get orthopheno paths
seed = list([
        'HGNC:17646',  # NGLY1 human gene
        'HGNC:633'  # AQP1 human gene
])
hypothesis.open_query(seed,queryname='ngly1_aqp1',port='7687')

Results!  

* outfile: query_ngly1_aqp1_paths_v2019-02-22.json

In [ ]:
%%time
import summary
# get summary
data = summary.path_load('./hypothesis/query_ngly1_aqp1_paths_v2019-02-22')

#parse data for summarization
data_parsed = list()
#funcs = [summary.metapaths, summary.nodes, summary.node_types, summary.edges, summary.edge_types]
for query in data:
    query_parsed = summary.query_parser(query)
    #metapath(query_parsed)
    #map(lambda x: x(query_parsed), funcs)
    data_parsed.append(query_parsed)
summary.metapaths(data_parsed)
summary.nodes(data_parsed)
summary.node_types(data_parsed)
summary.edges(data_parsed)
summary.edge_types(data_parsed)
#for query in data_parsed:
#    map(lambda x: x(query), funcs)

It seems that works. Only summaries for aqp1->ngly1. Check if it is correct. Also check the debugging part below what is it???